In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN

In [ ]:
train = pd.read_csv("../input/shopee-product-matching/train.csv")
train

In [ ]:
test = pd.read_csv("../input/shopee-product-matching/test.csv")
test

In [ ]:
train.title.nunique()

In [ ]:
train.shape

In [ ]:
len(os.listdir("../input/shopee-product-matching/train_images"))

In [ ]:
train.label_group.nunique()

In [ ]:
train[train['label_group']==249114794]

In [ ]:
img = plt.imread("../input/shopee-product-matching/train_images/0000a68812bc7e98c42888dfb1c07da0.jpg")
plt.imshow(img)

In [ ]:
img = plt.imread("../input/shopee-product-matching/train_images/f83b49a86a0ee8592e3bf0204da3fbdf.jpg")
plt.imshow(img)

In [ ]:
def show_group(df, label_group):
    tmp = df[df['label_group']==label_group]
    img_dir = "../input/shopee-product-matching/train_images"
    imgs_path = [ f"{img_dir}/{name}" for name in list(tmp['image']) ]
    imgs = [plt.imread(path) for path in imgs_path]
    titles = list(tmp['title'])
    for tt in titles:
        print(tt)
    cols = 4
    rows = (len(imgs)+cols-1)//cols
    plt.figure(figsize=(10, 10))
    for i in range(0,len(imgs)):
        plt.subplot(rows,cols,i+1)
        plt.imshow(imgs[i])
        plt.xticks([])
        plt.yticks([])
    plt.show()

In [ ]:
def show(tmp):
    img_dir = "../input/shopee-product-matching/train_images"
    imgs_path = [ f"{img_dir}/{name}" for name in list(tmp['image']) ]
    imgs = [plt.imread(path) for path in imgs_path]
    titles = list(tmp['title'])
    for tt in titles:
        print(tt)
    cols = 4
    rows = (len(imgs)+cols-1)//cols
    plt.figure(figsize=(10, 10))
    for i in range(0,len(imgs)):
        plt.subplot(rows,cols,i+1)
        plt.imshow(imgs[i])
        plt.xticks([])
        plt.yticks([])
    plt.show()

In [ ]:
show_group(train, 249114794)

In [ ]:
show_group(train, 2937985045)

In [ ]:
show_group(train, 2395904891)

In [ ]:
show_group(train, 4093212188)

In [ ]:
show_group(train , 3648931069)

In [ ]:
show_group(train, 3776555725)

In [ ]:
show_group(train , 2736479533)

In [ ]:
show_group(train , 1663538013)

In [ ]:
show_group(train , 1141798720)

In [ ]:
show_group(train, 994676122)

1. clustering image by image phash
2. clustering image by title similarity
3. using DBSCAN to clustering ?

In [ ]:
train[train['label_group']==1141798720]

In [ ]:
img = plt.imread("../input/shopee-product-matching/train_images/ffdd054398884b04bdab9e41d30c7fb2.jpg")
plt.imshow(img)

In [ ]:

img = plt.imread("../input/shopee-product-matching/train_images/6e5e87b82b0b484f7739f83a0de654f5.jpg")
plt.imshow(img)

In [ ]:
tmp = train[train['label_group']==1141798720]
tmp = tmp.reset_index()

In [ ]:
tmp

In [ ]:
def metric_fun(x1, x2):
    assert len(x1)==len(x2), f"{len(x1)}, {len(x2)}"
    length = len(x1)
    cnt = 0
    for i in range(length):
        if x1[i]!=x2[i]:
            cnt = cnt + 1
    return cnt

def str2ary(s):
    ret = []
    for i in range(len(s)):
        ret.append(ord(s[i]))
        
    return np.array(ret)

In [ ]:
X = np.array([str2ary(s) for s in tmp['image_phash'].values])


In [ ]:
X.shape

In [ ]:
labels = DBSCAN(eps=1, min_samples=2, metric=metric_fun).fit_predict(X)
labels

In [ ]:
tmp['cluster_label'] = labels

In [ ]:
show(tmp[tmp['cluster_label']==-1])

In [ ]:
show(tmp[tmp['cluster_label']==0])

In [ ]:
show(tmp[tmp['cluster_label']==1])

In [ ]:
show(tmp[tmp['cluster_label']==2])

In [ ]:
show(tmp[tmp['cluster_label']==3])

In [ ]:
show(tmp[tmp['cluster_label']==4])

In [ ]:
show(tmp[tmp['cluster_label']==5])

In [ ]:
train_smp = train.sample(n=1000).reset_index()
X_train = np.array([str2ary(s) for s in train_smp['image_phash'].values])
train_cluster_labels = DBSCAN(eps=3, min_samples=2, metric=metric_fun).fit_predict(X_train)
train_smp['cluster_labels']=train_cluster_labels
train_smp.cluster_labels.unique()

In [ ]:
show(train_smp[train_smp['cluster_labels']==0])

In [ ]:
show(train_smp[train_smp['cluster_labels']==-1].sample(n=20))

In [ ]:
show(train_smp[train_smp['cluster_labels']==-1].sample(n=20))


正负样本构造思路：  
正例：从同一个group中抽取正例样本对，根据image phash和文本距离，选取难例样本，丢弃简单样本对（距离非常近）  
负例：从不同的group中抽取负例样本对，根据image phash和文本距离，选取难例样本，丢弃简单样本对（距离非常远）  